In [4]:
# Import important libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from sklearn.metrics import mean_squared_error
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',100)
%matplotlib inline

In [5]:
# Import training and test data
train_data = pd.read_csv('Clean_data_train.csv')
test_data = pd.read_csv('Clean_data_test.csv')
print(train_data.shape)
print(test_data.shape)

(287816, 10)
(47860, 9)


In [6]:
train_data.head()

,Year,OG,Dept,Union,JF,Job,Salaries,Overtime,H/D,Total_Compensation
0,2015,Community Health,Public Health,"SEIU - Health Workers, Local 1021",Med Therapy & Auxiliary,Morgue Attendant,12196,0,0.00,16158
1,2013,Community Health,Public Health,"Stationary Engineers, Local 39",Journeyman Trade,Stationary Engineer,74639,2820,12703.31,115784
2,2015,General Administration & Finance,Assessor/Recorder,"Prof & Tech Engineers - Miscellaneous, Local 21",Appraisal & Taxation,Senior Real Property Appraiser,100554,0,12424.50,144708
3,2016,Public Protection,Police,Police Officers' Association,Police Services,Sergeant 3,140164,52754,13043.87,242323
4,2013,"Public Works, Transportation & Commerce",PUC Hetch Hetchy,"Prof & Tech Engineers - Miscellaneous, Local 21",Information Systems,IS Engineer-Journey,58813,0,7655.28,82106


In [7]:
# Taking Median value of Salaries , Overtime,and Insurance column as per Categories of Categorical Column
train_data['Median_Salary_As_Per_Dept'] = train_data.groupby('Dept')['Salaries'].transform('median')
train_data['Median_Overtime_As_Per_Dept'] = train_data.groupby('Dept')['Overtime'].transform('median')
train_data['Median_Insurance_As_Per_Dept'] = train_data.groupby('Dept')['H/D'].transform('median')

train_data['Median_Salary_As_Per_Union'] = train_data.groupby('Union')['Salaries'].transform('median')
train_data['Median_Overtime_As_Per_Union'] = train_data.groupby('Union')['Overtime'].transform('median')
train_data['Median_Insurance_As_Per_Union'] = train_data.groupby('Union')['H/D'].transform('median')

train_data['Median_Salary_As_Per_JF'] = train_data.groupby('JF')['Salaries'].transform('median')
train_data['Median_Overtime_As_Per_JF'] = train_data.groupby('JF')['Overtime'].transform('median')
train_data['Median_Insurance_As_Per_JF'] = train_data.groupby('JF')['H/D'].transform('median')

train_data['Median_Salary_As_Per_Job'] = train_data.groupby('Job')['Salaries'].transform('median')
train_data['Median_Overtime_As_Per_Job'] = train_data.groupby('Job')['Overtime'].transform('median')
train_data['Median_Insurance_As_Per_Job'] = train_data.groupby('Job')['H/D'].transform('median')

test_data['Median_Salary_As_Per_Dept'] = test_data.groupby('Dept')['Salaries'].transform('median')
test_data['Median_Overtime_As_Per_Dept'] = test_data.groupby('Dept')['Overtime'].transform('median')
test_data['Median_Insurance_As_Per_Dept'] = test_data.groupby('Dept')['H/D'].transform('median')

test_data['Median_Salary_As_Per_Union'] = test_data.groupby('Union')['Salaries'].transform('median')
test_data['Median_Overtime_As_Per_Union'] = test_data.groupby('Union')['Overtime'].transform('median')
test_data['Median_Insurance_As_Per_Union'] = test_data.groupby('Union')['H/D'].transform('median')

test_data['Median_Salary_As_Per_JF'] = test_data.groupby('JF')['Salaries'].transform('median')
test_data['Median_Overtime_As_Per_JF'] = test_data.groupby('JF')['Overtime'].transform('median')
test_data['Median_Insurance_As_Per_JF'] = test_data.groupby('JF')['H/D'].transform('median')

test_data['Median_Salary_As_Per_Job'] = test_data.groupby('Job')['Salaries'].transform('median')
test_data['Median_Overtime_As_Per_Job'] = test_data.groupby('Job')['Overtime'].transform('median')
test_data['Median_Insurance_As_Per_Job'] = test_data.groupby('Job')['H/D'].transform('median')

In [8]:
train_data.drop(labels = ['Dept', 'Union', 'JF', 'Job'], axis=1, inplace=True)
test_data.drop(labels = ['Dept', 'Union', 'JF', 'Job'], axis=1, inplace=True)

In [9]:
X_train = train_data.drop('Total_Compensation', axis=1)
y_train = train_data['Total_Compensation']
X_test = test_data

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)

(287816, 17)
(47860, 17)
(287816,)


## CatBoost

In [ ]:
from catboost import CatBoostRegressor
cat_boost = CatBoostRegressor(n_estimators=1000, random_state=101, learning_rate=0.05,max_depth=8, colsample_bylevel=1,
                             cat_features=['OG'])
cat_boost.fit(X_train, y_train)

y_train_prediction = cat_boost.predict(X_train)
y_test_prediction = cat_boost.predict(X_test)

In [13]:
print("Root Mean Squared Error on Training Set :" ,np.sqrt(mean_squared_error(y_train, y_train_prediction)))

Root Mean Squared Error on Training Set : 5542.897785696179


**Root_Mean_Squared_Error on Test Set : 6135.29**

## XGBoost

In [14]:
X_train = pd.get_dummies(X_train,columns=['OG'])
X_test = pd.get_dummies(X_test,columns=['OG'])

In [15]:
from xgboost import XGBRegressor

xg_boost = XGBRegressor(learning_rate=0.05,n_estimators=600,max_depth=9, colsample_bytree=1,colsample_bylevel=0.9,
                       colsample_bynode=0.9,booster='gbtree', random_state=101)
xg_boost.fit(X_train, y_train)

y_train_prediction = xg_boost.predict(X_train)
y_test_prediction = xg_boost.predict(X_test)

print("Root_Mean Squared Error on Training Set :" ,np.sqrt(mean_squared_error(y_train, y_train_prediction)))

Root_Mean Squared Error on Training Set : 4276.745049922959


**Root_Mean_Squared_Error on Test Set : 5881.80**

## LightGBM

In [18]:
from lightgbm import LGBMRegressor
import re

X_train = X_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
X_test = X_test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

lgb_regressor = LGBMRegressor(boosting_type='gbdt', max_depth=9, n_estimators=800,learning_rate=1/20, random_state=101)
lgb_regressor.fit(X_train, y_train)

y_train_prediction = lgb_regressor.predict(X_train)
y_test_prediction = lgb_regressor.predict(X_test)

print("Root Mean Squared Error on Training Set :" ,np.sqrt(mean_squared_error(y_train, y_train_prediction)))

Root Mean Squared Error on Training Set : 5472.466110489023


**Root_Mean_Squared_Error on Test Set : 6136.44**

In [19]:
sample_submission = pd.read_csv('sample_submission.csv')
sample_submission['Compensation'] = y_test_prediction
sample_submission.to_csv('sample_submission_1.csv', index=False)